
# EarthDaily Simulated Dataset

Following a pivot field


## Import librairies



In [ ]:
import datetime
import earthdaily
from dask.diagnostics import ProgressBar
from matplotlib import pyplot as plt
from earthdaily import datasets, EarthDataStore

ProgressBar().register()  # to have chunks progress bar

## Loading pivot



In [ ]:
pivot = datasets.load_pivot_corumba()

## Init earthdatastore with environment variables or default credentials



In [ ]:
eds = EarthDataStore()

## Define timerange



In [ ]:
delta_days = 10
datetime_list = ["2018-10-01", "2019-04-15"]

## Request items for vnir dataset



In [ ]:
items = eds.search(
    "earthdaily-simulated-cloudless-l2a-cog-edagro",
    intersects=pivot,
    datetime=datetime_list,
    query={"instruments": {"contains": "vnir"}},
    prefer_alternate="download",
)[::delta_days]  # an keep on item every n delta_days

## Generate datacube for RGB and NIR



In [ ]:
datacube = earthdaily.earthdatastore.datacube(
    items, intersects=pivot, assets=["blue", "green", "red", "nir"]
).load()

## Plot RGB image time series



In [ ]:
datacube[["red", "green", "blue"]].ed.plot_rgb(
    col="time", col_wrap=4, vmax=0.2
)

## Plot mean RGB time series



In [ ]:
datacube[["blue", "green", "red", "nir"]].groupby("time").mean(...).to_array(
    dim="band"
).plot(col="band")

## Plot NDVI evolution



In [ ]:
datacube["ndvi"] = (datacube["nir"] - datacube["red"]) / (
    datacube["nir"] + datacube["red"]
)

fig, ax = plt.subplots()
mean_ndvi = datacube[["ndvi"]].groupby("time").mean(...).to_array(dim="band")
std_ndvi = datacube[["ndvi"]].groupby("time").std(...).to_array(dim="band")
ax.fill_between(
    mean_ndvi.time,
    (mean_ndvi.values + std_ndvi.values)[0, ...],
    (mean_ndvi.values - std_ndvi.values)[0, ...],
    alpha=0.3,
    color="C1",
)
mean_ndvi.plot(ax=ax, c="C1")
plt.grid(alpha=0.4)
plt.title("NDVI evolution every 10 days")